<!-- This notebook is created by Siu Pui Cheung, 09/02/2024 -->

## 0. Utilities

In [18]:
import cv2, os, sys, numpy as np, pandas as pd, tkinter as tk
from tkinter import filedialog, PhotoImage
import mediapipe as mp
import threading
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
from datetime import datetime
plt.switch_backend('PDF') # Setup for matplotlib to use PDF backend

# Initialize mediapipe pose class
mp_pose, pose_landmark = mp.solutions.pose, mp.solutions.pose.PoseLandmark
landmarks_to_draw = list(mp_pose.PoseLandmark)
posture_data, buffer, sml_buffer, sign, run, stop_evaluation = {}, 10, 5, "| ", True, False

joint_angles_df = pd.DataFrame()
body_labels =[ ['Left shoulder', 'Right Shoulder', 'Left Elbow', 'Right Elbow', 'Left Wrist', 'Right Wrist', 'Left Upper Body', 'Right Upper Body', 'Left Knee', 'Right Knee', 'Left Low Limb', 'Right Low Limb', 'Shoulder Midpoint', 'Central Vertical Midpoint'],
    ['Left Shoulder', 'Right Shoulder', 'Left Elbow', 'Right Elbow', 'Left Upper Body', 'Right Upper Body', 'Left Knee', 'Right Knee', 'Left Low Limb', 'Right Low Limb', 'Left Neck', ' Right Neck', 'Left Hip', 'Right Hip', 'Left Ankle Flexion', 'Right Ankle Flexion'],
    ['Shoulder Angle Difference', 'Elbow Angle Difference', 'Hip Angle Difference'],
    ['Shoulder Angle Difference', 'Shoulder Midpoint X-coordinate', 'Central Vertical Line X-coordinate'],
    ['Ear Angle Difference', 'Shoulder Angle Difference', 'Hip Angle Difference', 'Knee Angle Difference', 'Ankle Angle Difference'],
    ['Left Shoulder Angle', 'Left Low Limb Angle']]

def gui(options):
    # Use os.getcwd() as a fallback when __file__ is not defined
    if getattr(sys, 'frozen', False):
        application_path = sys._MEIPASS  # When running as exe, the path is different.
    else:
        application_path = os.getcwd()  # Fallback to current working directory

    # Adjust the path to the background image using application_path
    bg_image_path = os.path.join(application_path, 'image/bg.png')  # Adjust this line for your images

    dialog = tk.Toplevel() # Initialize dialog window
    dialog.title("AI Posture Evaluator")

    # Load and place background image using the adjusted path
    bg_image = PhotoImage(file=bg_image_path).subsample(2, 2)
    tk.Label(dialog, image=bg_image).place(x=0, y=0, relwidth=1, relheight=1)
    dialog.bg_image = bg_image  # Keep a reference
    tk.Label(dialog, text="© 2024 E.C.| v1.1.1", font=("Helvetica", 5)).place(x=0, y=bg_image.height() - 10)
    dialog.geometry(f"{bg_image.width()}x{bg_image.height()}")
    dialog.grab_set()

    result = [None] # Store the user's selection
    params = 120, 120, 20, 10, 3  # button_width, button_height, space_between_buttons_x, space_between_buttons_y, num_columns
    total_width = (params[0] * params[4]) + (params[2] * (params[4] - 1))
    start_x, start_y = ((bg_image.width() - total_width) / 2), (bg_image.height() / 4)
    # Function to handle selection and close dialog
    def make_selection(value): result[0] = value; dialog.destroy()
    # Create and place buttons based on provided options
    for index, (text, value) in enumerate(options.items()):
        x, y = start_x + (index % params[4]) * (params[0] + params[2]), start_y + (index // params[4]) * (params[1] + params[3])
        tk.Button(dialog, text=text, command=lambda v=value: make_selection(v), font=("Helvetica", 10, "bold")).place(x=x, y=y, width=params[0], height=params[1])
        
    dialog.wait_window() # Wait for the dialog to close before continuing
    return result[0]

def initialize_capture():
    ROOT = tk.Tk()
    ROOT.withdraw()  # Hide tkinter root window
    # Mapping of analysis and detection functions
    ad_map = {1: (squat_front_analysis, squat_front_detection), 2: (squat_side_analysis, squat_side_detection),
              3: (squat_back_analysis, squat_back_detection), 4: (knee_raising_analysis, knee_raising_detection),
              5: (stand_front_analysis, stand_front_detection), 6: (stand_side_analysis, stand_side_detection)}

    a_opts = {"Front Angle": 1, "Side Angle": 2, "Balance Back": 3, "Balance Test": 4, "Balance Front": 5, "Balance Side": 6}
    i_opts = {"Image": 1, "Video file": 2, "Camera": 3}
    # User selects option or exits if none selected
    a_choice = gui(a_opts) or sys.exit(0)  # Exit if no choice
    anal_func, detect_func = ad_map[a_choice]
    i_choice = gui(i_opts) or sys.exit(0)  # Exit if no choice
    # File type filter based on input choice, skips if camera is selected
    file_type = [("Image files", "*.jpg *.jpeg *.png")] if i_choice == 1 else [("Video files", "*.mp4 *.mov *.avi")]
    path = filedialog.askopenfilename(title="Select the file", filetypes=file_type) if i_choice != 3 else None
    if i_choice != 3 and not path: sys.exit(0)  # Exit if no file selected
    # Open video capture: camera for option 3, file path otherwise
    cap = cv2.VideoCapture(0 if i_choice == 3 else path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS) if cap.isOpened() else 0
    return cap, i_choice == 1, path, frame_rate, anal_func, detect_func, a_choice-1

def setup_output_writer(cap, is_image, timestamp):
    ROOT = tk.Tk()
    ROOT.withdraw()  # Hide the root window
    output_folder = 'output'
    
    os.makedirs(output_folder, exist_ok=True)
    out_path = f"{output_folder}/Result_{timestamp}" + ('.jpg' if is_image else '.mp4')
    if not is_image:
        return cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'MP4V'), cap.get(cv2.CAP_PROP_FPS), (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
    return out_path

def process_frame(frame, pose, anal_func, detect_func):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    if results.pose_landmarks:
        # Get the posture and angles data from the analysis function
        posture, angles_data = anal_func(results.pose_landmarks.landmark, mp_pose)
        # Apply the detection function to annotate the image, also pass angles_data now
        annotated_image = detect_func(cv2.cvtColor(image, cv2.COLOR_RGB2BGR), results, posture, angles_data)
        return annotated_image, angles_data
    return cv2.cvtColor(image, cv2.COLOR_RGB2BGR), {}

def get_point(landmarks, landmark):
    return landmarks[landmark.value].x, landmarks[landmark.value].y

def calculate_angle(a, b, c, reverse=False):
    """
    Calculates the angle formed by three points, with an option to reverse the direction.

    Args:
    a, b, c (tuple): Coordinates of the points forming the angle (each as an (x, y) tuple).
    reverse (bool): If True, calculates the angle in the reverse direction.

    Returns:
    float: The calculated angle in degrees.
    """
    a, b, c = np.array(a), np.array(b), np.array(c)
    # Calculating the angle depending on the reverse flag
    if reverse:
        radians = np.arctan2(b[1] - a[1], b[0] - a[0]) - np.arctan2(c[1] - a[1], c[0] - a[0])
    else:
        radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.degrees(radians)
    # Adjusting angle to the range [0, 360] and then normalizing to [-180, 180]
    angle = (angle + 360) % 360
    if angle > 180:
        angle -= 360

    return angle

def draw_colored_connection(image, results, start_idx, end_idx, color=(255, 0, 0), thickness=2):

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        start, end = landmarks[start_idx], landmarks[end_idx]
        cv2.line(image, (int(start.x * image.shape[1]), int(start.y * image.shape[0])), 
                 (int(end.x * image.shape[1]), int(end.y * image.shape[0])), color, thickness)

def calculate_posture_ratio():
    total_postures = sum(sum(postures.values()) for section, postures in posture_data.items() if isinstance(postures, dict))
    correct_data = posture_data.get("Correct", 0)
    correct_count = sum(correct_data.values()) if isinstance(correct_data, dict) else correct_data

    section_percentages, top_level_percentages = {}, {}
    for section, postures in posture_data.items():
        if section != "Correct" and isinstance(postures, dict):
            section_total = sum(postures.values())
            section_percentages[section] = {cond: f"{(cnt / section_total) * 100:.1f}%" for cond, cnt in postures.items()}
            top_level_percentages[f"{section} Issue"] = f"{(section_total / total_postures) * 100:.1f}%"

    correct_ratio = f"{(correct_count / max(total_postures, 1)) * 100:.1f}%"
    top_level_percentages["Correct Ratio"] = correct_ratio

    return {**section_percentages, **top_level_percentages}


def update_posture_data(section, posture_conditions, detected_postures):
    if section == "Correct":
        posture_data[section] = {"Correct": posture_data.get(section, {}).get("Correct", 0) + 1}
    else:
        detected_postures_list = [p.strip() for p in detected_postures.split(" and ") if p.strip() in posture_conditions]
        for posture in detected_postures_list:
            posture_data.setdefault(section, {}).setdefault(posture, 0)
            posture_data[section][posture] += 1

            
            
def draw_landmarks(image, results, landmark_indices, color=(255, 255, 255), radius=3):

    for idx in landmark_indices:
        landmark_point = results.pose_landmarks.landmark[idx]
        pos = (int(landmark_point.x * image.shape[1]), int(landmark_point.y * image.shape[0]))
        cv2.circle(image, pos, radius, color, -1)

            
            
def draw_labeled_box(image, results, joint_landmarks, angles, padding=3, font_scale=0.35, font_thickness=1,
                     box_color=(255, 255, 255), text_color=(139, 0, 0), edge_color=(230, 216, 173)): 
    for joint_index, angle in enumerate(angles):
        joint = results.pose_landmarks.landmark[joint_landmarks[joint_index]]
        angle_text, pos = f"{round(angle)}", (int(joint.x * image.shape[1]) + 10, int(joint.y * image.shape[0]))
        text_size = cv2.getTextSize(angle_text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)[0]
        box_start, box_end = (pos[0] - padding, pos[1] + padding), (pos[0] + text_size[0] + padding, pos[1] - text_size[1] - padding)
        cv2.rectangle(image, box_start, box_end, box_color, cv2.FILLED)
        cv2.rectangle(image, box_start, box_end, edge_color, 1)
        cv2.putText(image, angle_text, (pos[0], pos[1]), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness, cv2.LINE_AA)


def generate_report(joint_angles_df, frame_rate, body_labels, analysis_choice, timestamp):
    joint_angles_df['Time'] = joint_angles_df.index / frame_rate
    joints = joint_angles_df.columns.drop('Time').tolist()

    if len(body_labels[analysis_choice]) != len(joints):
        # Plot "No Posture detected." message instead of raising error
        fig = plt.figure(figsize=(10, 6))
        plt.suptitle('Analysis Report', fontsize=20)
        plt.text(0.5, 0.5, 'No Posture detected.', fontsize=14, ha='center')
        plt.axis('off')
        os.makedirs('output', exist_ok=True)
        plt.savefig(os.path.join('output', f"Report_{timestamp}.pdf"))
        plt.close(fig)
        return

    num_rows, fig_width, fig_height = ((len(joints) + 1) // 2) + 5, 18, (((len(joints) + 1) // 2) + 5) * 4
    fig = plt.figure(figsize=(fig_width, fig_height))
    fig.suptitle('Analysis Report', fontsize=20, y=1)
    gs = GridSpec(num_rows, 4, figure=fig, width_ratios=[3, 1, 3, 1])

    for i, joint in enumerate(joints):
        row, col = i // 2, (i % 2) * 2
        ax_plot = fig.add_subplot(gs[row, col])
        ax_plot.plot(joint_angles_df['Time'], joint_angles_df[joint], label=joint)
        ax_plot.set_title(body_labels[analysis_choice][i])
        ax_plot.set_xlabel('Time (s)')
        ax_plot.set_ylabel('Angle (degrees)')
        ax_plot.grid(True)

        stats = { 'Average': round(np.mean(joint_angles_df[joint]), 2),
                  'Max': round(np.max(joint_angles_df[joint]), 2),
                  'Min': round(np.min(joint_angles_df[joint]), 2) }
        ax_table = fig.add_subplot(gs[row, col + 1])
        ax_table.axis('off')
        table = ax_table.table(cellText=[[k, f'{v:.2f}'] for k, v in stats.items()], colLabels=['Stat', 'Value'], cellLoc='center', loc='center')
        table.auto_set_font_size(True)
        table.scale(1, 1.5)

    plt.subplots_adjust(hspace=1)
    fig.tight_layout()
    os.makedirs('output', exist_ok=True)
    plt.savefig(os.path.join('output', f"Report_{timestamp}.pdf"))
    plt.close(fig)
    
def run_estimation():
    global stop_evaluation, joint_angles_df
    stop_evaluation, joint_angles_df = False, pd.DataFrame()
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    cap, is_image, _, frame_rate, anal_func, detect_func, analysis_choice = initialize_capture()
    if not cap: return

    out = setup_output_writer(cap, is_image, timestamp)

    # Start the stop/proceed GUI in a separate thread
    gui_thread = threading.Thread(target=show_stop_gui, daemon=True)
    gui_thread.start()

    with mp.solutions.pose.Pose(min_detection_confidence=0.6 if is_image else 0.9, min_tracking_confidence=0.6 if is_image else 0.9) as pose:
        while cap.isOpened() and not stop_evaluation:
            ret, frame = cap.read()
            if not ret: break

            processed_frame, angles_data = process_frame(frame, pose, anal_func, detect_func)
            joint_angles_df = pd.concat([joint_angles_df, pd.DataFrame([angles_data])], ignore_index=True)
            cv2.imshow('Mediapipe Feed', processed_frame)
            if is_image: cv2.imwrite(out, processed_frame); break
            elif out: out.write(processed_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'): break

    cap.release()
    cv2.destroyAllWindows()
    if out and not is_image: out.release()
    gui_thread.join()  # Wait for the user to click 'Proceed' before moving on
    posture_percentages = calculate_posture_ratio()
    df = pd.DataFrame([{'Section': s, 'Condition': c, 'Percentage': p} for s, cs in posture_percentages.items() for c, p in (cs.items() if isinstance(cs, dict) else [(None, cs)])])
    generate_report(joint_angles_df, frame_rate, body_labels, analysis_choice, timestamp)
    return df


def show_stop_gui():
    global stop_evaluation
    
    def on_button_click():
        if button['text'] == 'Stop':
            global stop_evaluation
            stop_evaluation = True  # Signal to stop the current analysis
            button.config(text='Proceed')  # Change button text to 'Proceed' after stopping
        else:
            root.destroy()  # Close the GUI and proceed to next round

    root = tk.Tk()
    root.title("Analysis Control")
    tk.Label(root, text="Click 'Stop' to stop analysis \nClick 'Proceed' to complete analysis.").pack(pady=20)
    
    button = tk.Button(root, text="Stop", command=on_button_click)
    button.pack(pady=10)
    
    root.protocol("WM_DELETE_WINDOW", root.destroy)  # Ensure the window can be closed directly
    root.mainloop()

    return stop_evaluation

def main():
    while run:
        run_estimation()

## 1.1 Squat Front

### 1.1.1 Analysis

In [2]:
def squat_front_analysis(landmarks, mp_pose):
    """
    Analyzes front-view squat posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing the detected posture and a tuple of angles (left knee angle, right knee angle, hip angle difference).
    """

    # Get points
    left_shoulder, left_elbow, left_wrist, left_index, left_hip, left_knee, left_ankle, left_foot_index = map(lambda lm: get_point(landmarks, lm), 
                                                                                                  [landmarks_to_draw[11], landmarks_to_draw[13], landmarks_to_draw[15], 
                                                                                                  landmarks_to_draw[23], landmarks_to_draw[25], landmarks_to_draw[27],
                                                                                                  landmarks_to_draw[31], landmarks_to_draw[19]])
    right_shoulder, right_elbow, right_wrist, right_index, right_hip, right_knee, right_ankle, right_foot_index = map(lambda lm: get_point(landmarks, lm), 
                                                                                                  [landmarks_to_draw[12], landmarks_to_draw[14], landmarks_to_draw[16], 
                                                                                                  landmarks_to_draw[24], landmarks_to_draw[26], landmarks_to_draw[28],
                                                                                                  landmarks_to_draw[32], landmarks_to_draw[20]])
    
    
    # Calculate shoulder angles
    left_shoulder_angle = abs(calculate_angle(left_hip, left_shoulder, left_elbow))
    right_shoulder_angle = abs(calculate_angle(right_hip, right_shoulder, right_elbow))
    
    # Calculate elbow angles
    left_elbow_angle = abs(calculate_angle(left_shoulder, left_elbow, left_wrist))
    right_elbow_angle = abs(calculate_angle(right_shoulder, right_elbow, right_wrist))
    
    # Calculate wrist angles
    left_wrist_angle = abs(calculate_angle(left_elbow, left_wrist, left_index, True))
    right_wrist_angle = abs(calculate_angle(right_elbow, right_wrist, right_index, True))
    
    # Calculate hip angles
    left_hip_angle = abs(calculate_angle(left_shoulder, left_hip, left_knee))
    right_hip_angle = abs(calculate_angle(right_shoulder, right_hip, right_knee))
    
    # Calculate knee angles
    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
    right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
    
    # Calculate ankle angles
    left_ankle_angle = abs(calculate_angle(left_ankle, left_foot_index, left_knee, True))
    right_ankle_angle = abs(calculate_angle(right_ankle, right_foot_index, right_knee, True))
    
    # Calcuate hip angles
    left_hip1_angle = abs(calculate_angle(right_hip, left_hip, [left_hip[0], 0]))
    right_hip1_angle = abs(calculate_angle(left_hip, right_hip, [right_hip[0], 0]))
    
    # Determine if hips are level
    left_hip_level = (left_hip_angle - right_hip_angle) > sml_buffer
    right_hip_level = (right_hip_angle - left_hip_angle) > sml_buffer
    
    
    # Calculate the midpoint of the shoulders
    shoulder_mid_x = (left_shoulder[0] + right_shoulder[0]) / 2
    # Calculate the central vertical line (midpoint between hips)
    central_vertical_line_x = (left_hip[0] + right_hip[0]) / 2
    
    
    correct_degree = 110
    
    # Outward position logic
    left_knee_outward = (left_knee_angle + correct_degree) > buffer
    right_knee_outward = (right_knee_angle - correct_degree) < -buffer

    # Inward position logic
    left_knee_inward = (left_knee_angle + correct_degree) < -buffer
    right_knee_inward = (right_knee_angle - correct_degree) > buffer

    
    # update posture condition
    knee_postures = ["Knees outwards", "Knees inwards"]
    hip_postures = ["H.L. hip", "H.R. hip"]

    posture = ""
    if (left_knee_outward or right_knee_outward) and (abs(left_knee_angle) < correct_degree + buffer and abs(right_knee_angle) < correct_degree + buffer):
        posture += sign + knee_postures[0]
    elif (left_knee_inward or right_knee_inward) and (abs(left_knee_angle) < correct_degree + buffer and abs(right_knee_angle) < correct_degree + buffer):
        posture += sign + knee_postures[1]
    if left_hip_level:
        posture += sign + hip_postures[0]
    elif right_hip_level:
        posture += sign + hip_postures[1]
    if posture == "":
        posture += "| Correct"

    detected_postures = posture.split(sign)
    for detected_posture in detected_postures:
        detected_posture = detected_posture.strip()
        if detected_posture in knee_postures:
            update_posture_data("Knee", knee_postures, detected_posture)
        elif detected_posture in hip_postures:
            update_posture_data("Hip", hip_postures, detected_posture)
        elif detected_posture == "Correct":
            posture_data["Correct"] = posture_data.get("Correct", 0) + 1

    # Return posture and angles as a tuple
    return posture, (left_shoulder_angle, right_shoulder_angle, left_elbow_angle, right_elbow_angle, left_wrist_angle, 
                     right_wrist_angle, left_hip_angle, right_hip_angle, abs(left_knee_angle),  abs(right_knee_angle), 
                     left_ankle_angle, right_ankle_angle, shoulder_mid_x, central_vertical_line_x)


### 1.1.2 Detection

In [3]:
def squat_front_detection(image, results, knee_position, angles):
    """
    Analyzes and annotates a front-view image of a squatting posture.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - knee_position (str): Description of the knee position (e.g., "Correct", "Too forward").
    - angles (tuple): A tuple containing the angles (left knee angle, right knee angle, hip angle difference).

    Returns:
    - numpy.ndarray: The annotated image with landmarks, angles, and posture information.
    """
    # Unpack angles to get the knee angles and hip angle difference
    if results.pose_landmarks:

        
        
        # Draw the lines
        connect_idx = [(23, 25), (25, 31), (27, 31), (24, 26), (26, 32), (28, 32), (11, 13), (13, 15),
                        (12, 14), (14, 16), (11, 23), (12, 24), (11, 12), (23, 24)]
        colors = [(255, 0, 0)] * 12 + [(0, 0, 255)] * 4
        for (p1, p2), color in zip(connect_idx, colors):
            draw_colored_connection(image, results, landmarks_to_draw[p1], landmarks_to_draw[p2], color=color)
        
        # Draw the specified landmarks
        landmark_idx = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 31, 32, 27, 28]
        draw_landmarks(image, results, landmark_idx) 
        joint_landmarks = [landmarks_to_draw[idx].value for idx in landmark_idx] # Draw the lines and display angles
        draw_labeled_box(image, results, joint_landmarks, angles[:-2]) # Call the draw_labeled_box function
        
        # central line
        central_vertical_line_x, shoulder_mid_x = angles[-1], angles[-2]
        # Draw central vertical line
        central_vertical_line_pixel_x = int(central_vertical_line_x * image.shape[1])
        cv2.line(image, (central_vertical_line_pixel_x, 0), (central_vertical_line_pixel_x, image.shape[0]), (255, 255, 0), 1)

        # Calculate middle dot position (midpoint between shoulders)
        middle_dot_x = int(shoulder_mid_x * image.shape[1])
        middle_dot_y = int((results.pose_landmarks.landmark[landmarks_to_draw[landmark_idx[0]]].y +
                            results.pose_landmarks.landmark[landmarks_to_draw[landmark_idx[1]]].y) / 2 * image.shape[0])
        
        # Draw the middle dot
        cv2.circle(image, (middle_dot_x, middle_dot_y), 3, (0, 255, 0), -1)  # Green dot
        shoulder_midpoint_pos = (middle_dot_x, middle_dot_y)
        # For "Shoulder Diff" text
        deviation_text = 'Left' if (shoulder_mid_x - central_vertical_line_x) > 0 else 'Right' if (shoulder_mid_x - central_vertical_line_x) < 0 else 'Centered'
        deviation_full_text = f'Dev: {deviation_text}'

        # Calculate size of the text for background box calculation
        deviation_text_size = cv2.getTextSize(deviation_full_text, cv2.FONT_HERSHEY_COMPLEX , 0.4, 2)[0]
        deviation_box_start = (central_vertical_line_pixel_x + 5, shoulder_midpoint_pos[1] + 15 - deviation_text_size[1] - 2)
        deviation_box_end = (deviation_box_start[0] + deviation_text_size[0] + 4, shoulder_midpoint_pos[1] + 15 + 2)
        cv2.rectangle(image, deviation_box_start, deviation_box_end, (255, 255, 255), cv2.FILLED)
        cv2.rectangle(image, deviation_box_start, deviation_box_end, (230, 216, 173), 1)
        # put the text on top of the boxes
        cv2.putText(image, deviation_full_text, (deviation_box_start[0], deviation_box_start[1] + deviation_text_size[1] + 2), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (139, 0, 0), 1)
        

        # Add text for knee position if necessary
        cv2.putText(image, f'Posture: {knee_position}', (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
    
    return image


## 1.2 Squat Side

### 1.2.1 Analysis

In [4]:
def squat_side_analysis(landmarks, mp_pose):
    """
    Analyzes front-view squat posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing the detected posture and a tuple of angles (left knee angle, right knee angle, hip angle difference).
    """
    # Get points
    left_shoulder, left_elbow, left_wrist, left_hip, left_knee, left_ankle, left_foot_index, left_ear = map(lambda lm: get_point(landmarks, lm), 
                                                                                                  [landmarks_to_draw[11], landmarks_to_draw[13], landmarks_to_draw[15], 
                                                                                                  landmarks_to_draw[23], landmarks_to_draw[25], landmarks_to_draw[27],
                                                                                                  landmarks_to_draw[31], landmarks_to_draw[7]])
    right_shoulder, right_elbow, right_wrist, right_hip, right_knee, right_ankle, right_foot_index, right_ear = map(lambda lm: get_point(landmarks, lm), 
                                                                                                  [landmarks_to_draw[12], landmarks_to_draw[14], landmarks_to_draw[16], 
                                                                                                  landmarks_to_draw[24], landmarks_to_draw[26], landmarks_to_draw[28],
                                                                                                  landmarks_to_draw[32], landmarks_to_draw[8]])

    
    
        
    # Analyse hip posture
    # Calculate hip angles
    left_hip_angle = abs(calculate_angle(left_shoulder, left_hip, [left_hip[0], 0]))
    right_hip_angle = abs(calculate_angle(right_shoulder, right_hip, [right_hip[0], 0]))
    
    # Calculate elbow angles
    left_elbow_angle = abs(calculate_angle(left_shoulder, left_elbow, left_wrist))
    right_elbow_angle = abs(calculate_angle(right_shoulder, right_elbow, right_wrist))
    
    # Calculate shoulder angles
    left_shoulder_angle = abs(calculate_angle(left_elbow, left_shoulder, left_hip))
    right_shoulder_angle = abs(calculate_angle(right_elbow, right_shoulder, right_hip))
    
    # Calculate torso angles
    left_torso_angle = abs(calculate_angle(left_shoulder, left_hip, left_knee))
    right_torso_angle = abs(calculate_angle(right_shoulder, left_hip, right_knee))
    
    # Calculate ankle front angles
    left_ankle_f_angle = abs(calculate_angle(left_knee, left_ankle, left_foot_index))
    right_ankle_f_angle = abs(calculate_angle(right_knee, right_ankle, right_foot_index))
    
    
     # Calculate neck angle
    left_neck_angle = abs(calculate_angle([left_shoulder[0], 0], left_shoulder, left_ear))
    right_neck_angle = abs(calculate_angle([right_shoulder[0], 0], right_shoulder, right_ear))

    # Determine if shoulder angle is correct
    correct_neck_angle = 0  # Correct shoulder angle (ideal posture)
    neck_forward = (left_neck_angle > (correct_neck_angle + buffer)) or (right_neck_angle > (correct_neck_angle + buffer))
    
    # Analyse ankle posture
    # Calculate ankle angles
    left_ankle_angle = abs(calculate_angle(left_knee, left_ankle, [left_ankle[0], 0]))
    right_ankle_angle = abs(calculate_angle(right_knee, right_ankle, [right_ankle[0], 0]))
    
    # Determine if hip-shoulder and ankle-knee lines are asymmetric
    left_asymmetric = abs(left_hip_angle - left_ankle_angle) > buffer + 15
    right_asymmetric = abs(right_hip_angle - right_ankle_angle) > buffer + 15
    
    
    # Analyse knee posture
    # Calculate knee angles, subtract 180 for reflection
    left_knee_angle = abs(calculate_angle(left_hip, left_knee, left_ankle))
    right_knee_angle = abs(calculate_angle(right_hip, right_knee, right_ankle))
    
    # Determine if knee angles are correct
    correct_knee_angle = 38
    knee_low = any([abs(left_knee_angle) > (correct_knee_angle - buffer), abs(right_knee_angle) > (correct_knee_angle - buffer)])

    # update posture condition
    asym_postures = ["L. body asym", "R. body asym"]
    knee_postures = ["Low knees"]
    neck_postures = ["Neck forward"]
    
    posture = ""
    if left_asymmetric:
        posture += sign + asym_postures[0]
    if right_asymmetric:
        posture += sign + asym_postures[1]
    if knee_low and (abs(left_knee_angle) < correct_knee_angle and abs(right_knee_angle) < correct_knee_angle):
        posture += sign + knee_postures[0]
    if neck_forward:
        posture += sign + neck_postures[0]
    if posture == "":
        posture = "Correct"
        
    detected_postures = posture.split(sign)
    for detected_posture in detected_postures:
        detected_posture = detected_posture.strip()
        if detected_posture in asym_postures:
            update_posture_data("Symmetry", asym_postures, detected_posture)
        elif detected_posture in knee_postures:
            update_posture_data("Knee", knee_postures, detected_posture)
        elif detected_posture in neck_postures:
            update_posture_data("Neck", neck_postures, detected_posture)
        elif detected_posture == "Correct":
            posture_data["Correct"] = posture_data.get("Correct", 0) + 1

        
    return posture, (left_shoulder_angle, right_shoulder_angle, left_elbow_angle, right_elbow_angle, 
                     left_hip_angle, right_hip_angle, left_knee_angle, right_knee_angle, left_ankle_angle, 
                     right_ankle_angle, left_neck_angle, right_neck_angle, left_torso_angle, right_torso_angle,
                     left_ankle_f_angle, right_ankle_f_angle)

### 1.2.2 Detection

In [5]:
def squat_side_detection(image, results, posture, angles):
    """
    Analyzes and annotates a side-view image of a squatting posture.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - posture (str): Description of the overall posture.
    - angles (list): A list of knee angles.
    - body_tilts (list): List of body tilts (left and right hip and ankle angles).

    Returns:
    - numpy.ndarray: The annotated image with landmarks and angles.
    """
    if results.pose_landmarks:
        
        
        # Draw the lines with specified colors
        connect_idx = [(7, 11), (8, 12), (11, 23), (25, 27), (12, 24), (26, 28), (11, 13), (13, 15), 
               (12, 14), (14, 16), (23, 25), (27, 31), (24, 26), (28, 32)]
    
        for i, (p1, p2) in enumerate(connect_idx):
            color = (0,165,255) if i < 2 else ((0, 0, 255) if i < 6 else (255, 0, 0))
            draw_colored_connection(image, results, landmarks_to_draw[p1], landmarks_to_draw[p2], color=color)
        
        # Draw the specified landmarks
        landmark_idx = [11, 12, 13, 14, 23, 24, 25, 26, 27, 28, 7, 8, 15, 16, 31, 32]
        draw_landmarks(image, results, landmark_idx) 
        joint_landmarks = [landmarks_to_draw[idx].value for idx in landmark_idx]
        draw_labeled_box(image, results, joint_landmarks, angles[:-4]) # Call the draw_labeled_box function
        

        
        # Find pose direction
        left_foot_index_x, right_foot_index_x = results.pose_landmarks.landmark[31].x, results.pose_landmarks.landmark[32].x
        left_ankle_x, right_ankle_x = results.pose_landmarks.landmark[27].x, results.pose_landmarks.landmark[28].x
        if left_foot_index_x < left_ankle_x or right_foot_index_x < right_ankle_x:
            direction = 1
        else:
            direction = -1

            
            
        # Draw box and text for torsos and angles
        landmark_idx_extra = [23, 24, 27, 28]
        joint_names_extra = ['L Torso', 'R Torso', 'L Ankle', 'R Ankle']
        joint_landmarks_extra = [landmarks_to_draw[idx].value for idx in landmark_idx_extra]
        for joint_index, angle in enumerate(angles[-4:]):
            joint_landmark = results.pose_landmarks.landmark[joint_landmarks_extra[joint_index]]

            angle_text = f"{round(angle)}"
            text_x = int(joint_landmark.x * image.shape[1]) - (90 * direction)
            text_y = int(joint_landmark.y * image.shape[0])

            # Determine the size of the text box
            text_size, _ = cv2.getTextSize(angle_text, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
            text_width, text_height = text_size

            box_start = (text_x - 2, text_y + 2)
            box_end = (text_x + text_width + 2, text_y - text_height - 2)

            # Draw the filled rectangle (background)
            cv2.rectangle(image, box_start, box_end, (255, 255, 255), cv2.FILLED)
            # Draw the border rectangle (edges)
            cv2.rectangle(image, box_start, box_end, (230, 216, 173), 1)

            # Now put the text (in specified text color)
            text_org = (text_x, text_y)
            cv2.putText(image, angle_text, text_org, cv2.FONT_HERSHEY_SIMPLEX, 0.35, (139, 0, 0), 1, cv2.LINE_AA)

        # Add text for knee position if necessary
        cv2.putText(image, f'Posture: {posture}', (0, 20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

    return image


## 1.3 Squat Back 

### 1.3.1 Analysis

In [6]:
def squat_back_analysis(landmarks, mp_pose):
    """
    Analyzes back-view squat posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing the detected posture and a tuple of angles (shoulder angle difference, hip angle difference).
    """

    # Get points
    left_shoulder, left_hip, left_elbow = map(lambda lm: get_point(landmarks, lm), [pose_landmark.LEFT_SHOULDER, pose_landmark.LEFT_HIP, pose_landmark.LEFT_ELBOW])
    right_shoulder, right_hip, right_elbow = map(lambda lm: get_point(landmarks, lm), [pose_landmark.RIGHT_SHOULDER, pose_landmark.RIGHT_HIP, pose_landmark.RIGHT_ELBOW])
    
    # Posture Analysis
    # Calculate shoulder angles
    left_shoulder_angle = abs(calculate_angle(right_shoulder, left_shoulder, [left_shoulder[0], 0]))
    right_shoulder_angle = abs(calculate_angle(left_shoulder, right_shoulder, [right_shoulder[0], 0]))
    
    # Determine if shoulders are level
    left_shoulder_high = left_shoulder_angle - right_shoulder_angle > sml_buffer
    right_shoulder_high = right_shoulder_angle - left_shoulder_angle > sml_buffer

     # Calculate hip angles
    left_hip_angle = abs(calculate_angle(right_hip, left_hip, [left_hip[0], 0]))
    right_hip_angle = abs(calculate_angle(left_hip, right_hip, [right_hip[0], 0]))
    
    # Determine if hips are level
    left_hip_high = left_hip_angle - right_hip_angle > sml_buffer
    right_hip_high = right_hip_angle - left_hip_angle > sml_buffer
    
    # Calculate elbow angles
    left_elbow_angle = abs(calculate_angle(right_elbow, left_elbow, [left_elbow[0], 0]))
    right_elbow_angle = abs(calculate_angle(left_elbow, right_elbow, [right_elbow[0], 0]))
    
    # Determine if elbows are level
    left_elbow_high = left_elbow_angle - right_elbow_angle > sml_buffer
    right_elbow_high = right_elbow_angle - left_elbow_angle > sml_buffer
    

    # Update posture condition
    shoulder_postures = ["H.L. shoulder", "H.R. shoulder"]
    hip_postures = ["H.L. hip", "H.R. hip"]
    posture = ""
    if left_shoulder_high:
        posture += sign + shoulder_postures[0]
    elif right_shoulder_high:
        posture += sign + shoulder_postures[1]
    if left_hip_high:
        posture += sign + hip_postures[0]
    elif right_hip_high:
        posture += sign + hip_postures[1]
    if posture == "":
        posture = "Correct"

    detected_postures = posture.split(sign)
    for detected_posture in detected_postures:
        detected_posture = detected_posture.strip()
        if detected_posture in shoulder_postures:
            update_posture_data("Shoulder", shoulder_postures, detected_posture)
        elif detected_posture in hip_postures:
            update_posture_data("Hip", hip_postures, detected_posture)
        elif detected_posture == "Correct":
            posture_data["Correct"] = posture_data.get("Correct", 0) + 1     
        
        
    return posture, (left_shoulder_angle - right_shoulder_angle, left_elbow_angle - right_elbow_angle, left_hip_angle - right_hip_angle)

### 1.3.2 Detection

In [7]:
def squat_back_detection(image, results, posture, angles):
    if results.pose_landmarks:
        landmark_idx = [11, 12, 13, 14, 23, 24]
        # Draw the lines
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[0]], landmarks_to_draw[landmark_idx[1]])
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[2]], landmarks_to_draw[landmark_idx[3]])
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[4]], landmarks_to_draw[landmark_idx[5]])
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[0]], landmarks_to_draw[landmark_idx[4]], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[1]], landmarks_to_draw[landmark_idx[5]], color=(0, 0, 255))

        
        draw_landmarks(image, results, landmark_idx) # Draw the specified landmarks

        joint_landmarks = [landmarks_to_draw[idx].value for idx in landmark_idx[::2]] # Draw the lines and display angles
        draw_labeled_box(image, results, joint_landmarks, angles) # Call the draw_labeled_box function

        # Add text for posture
        cv2.putText(image, f'Posture: {posture}', (20, 50), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2)

    return image


## 2. Knee Raising

### 2.0.1 Analysis

In [8]:
def knee_raising_analysis(landmarks, mp_pose):
    """
    Analyzes posture during knee-raising exercises based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing the detected posture and a list with shoulder angle difference, shoulder midpoint x-coordinate, and central vertical line x-coordinate.
    """
    # Get points for shoulders and hips
    left_shoulder, left_hip = map(lambda lm: get_point(landmarks, lm), [pose_landmark.LEFT_SHOULDER, pose_landmark.LEFT_HIP])
    right_shoulder, right_hip = map(lambda lm: get_point(landmarks, lm), [pose_landmark.RIGHT_SHOULDER, pose_landmark.RIGHT_HIP])
    
    
    
    # Posture Analysis
    # Calculate shoulder angles
    left_shoulder_angle = abs(calculate_angle(right_shoulder, left_shoulder, [left_shoulder[0], 0]))
    right_shoulder_angle = abs(calculate_angle(left_shoulder, right_shoulder, [right_shoulder[0], 0]))
    
    # Determine if shoulders are level
    left_shoulder_high = left_shoulder_angle - right_shoulder_angle > sml_buffer
    right_shoulder_high = right_shoulder_angle - left_shoulder_angle > sml_buffer

    
    
    # Calculate the midpoint of the shoulders
    shoulder_mid_x = (left_shoulder[0] + right_shoulder[0]) / 2
    # Calculate the central vertical line (midpoint between hips)
    central_vertical_line_x = (left_hip[0] + right_hip[0]) / 2
    
    mp_buffer = 0.025 # set buffer for midpoint
    # Determine if the shoulder mid point is over the left or right of the hip mid point
    mid_point_left = (shoulder_mid_x - central_vertical_line_x) < -mp_buffer
    mid_point_right = (shoulder_mid_x - central_vertical_line_x) > mp_buffer
    # Update posture condition
    shoulder_postures = ["H.L. shoulder", "H.R. shoulder"]
    mp_postures = ["Body too left", "Body too right"]
    
    posture = ""
    if left_shoulder_high:
        posture += sign + shoulder_postures[0]
    elif right_shoulder_high:
        posture += sign + shoulder_postures[1]
    if mid_point_left:
        posture += sign + mp_postures[0]
    elif mid_point_right:
        posture += sign + mp_postures[1]
    if posture == "":
        posture = "Correct"

    detected_postures = posture.split(sign)
    for detected_posture in detected_postures:
        detected_posture = detected_posture.strip()
        if detected_posture in shoulder_postures:
            update_posture_data("Shoulder", shoulder_postures, detected_posture)
        elif detected_posture in mp_postures:
            update_posture_data("Mid Point", mp_postures, detected_posture)
        elif detected_posture == "Correct":
            posture_data["Correct"] = posture_data.get("Correct", 0) + 1 

    return posture, (left_shoulder_angle - right_shoulder_angle, shoulder_mid_x, central_vertical_line_x)

### 2.0.2 Detection

In [9]:
def knee_raising_detection(image, results, posture, angles):
    """
    Analyzes and annotates an image for knee raising exercises.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - posture (str): Description of the overall posture.
    - angles (tuple): Contains analysis data like shoulder vertical difference,
                                shoulder midpoint, and central vertical line position.

    Returns:
    - numpy.ndarray: The annotated image with landmarks, vertical line, and deviation information.
    """

    shoulder_vertical_difference, shoulder_mid_x, central_vertical_line_x = angles

    if results.pose_landmarks:
        landmark_idx = [11, 12, 23, 24]
        # Draw the lines
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[0]], landmarks_to_draw[landmark_idx[1]])
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[2]], landmarks_to_draw[landmark_idx[3]], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[0]], landmarks_to_draw[landmark_idx[2]], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[1]], landmarks_to_draw[landmark_idx[3]], color=(0, 0, 255))

        
        # Draw central vertical line
        central_vertical_line_x, shoulder_mid_x = angles[-1], angles[-2]
        # Draw central vertical line
        central_vertical_line_pixel_x = int(central_vertical_line_x * image.shape[1])
        cv2.line(image, (central_vertical_line_pixel_x, 0), (central_vertical_line_pixel_x, image.shape[0]), (255, 255, 0), 1)
        # Calculate middle dot position (midpoint between shoulders)
        middle_dot_x = int(shoulder_mid_x * image.shape[1])
        middle_dot_y = int((results.pose_landmarks.landmark[landmarks_to_draw[landmark_idx[0]]].y +
                            results.pose_landmarks.landmark[landmarks_to_draw[landmark_idx[1]]].y) / 2 * image.shape[0])
        draw_landmarks(image, results, landmark_idx)
        # Draw the middle dot
        cv2.circle(image, (middle_dot_x, middle_dot_y), 3, (0, 255, 0), -1)  # Green dot
        
        # Calculate positions for drawing text
        shoulder_midpoint_pos = (middle_dot_x, middle_dot_y)
    
        # For "Shoulder Diff" text
        shoulder_diff_text = f'Diff: {round(abs(shoulder_vertical_difference), 1)}'
        deviation_text = 'Left' if (shoulder_mid_x - central_vertical_line_x) > 0 else 'Right' if (shoulder_mid_x - central_vertical_line_x) < 0 else 'Centered'
        deviation_full_text = f'Dev: {deviation_text}'

        # Calculate size of the text for background box calculation
        shoulder_diff_text_size = cv2.getTextSize(shoulder_diff_text, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)[0]
        deviation_text_size = cv2.getTextSize(deviation_full_text, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)[0]

        # Shoulder Diff Box
        shoulder_diff_box_start = (shoulder_midpoint_pos[0] + 5, shoulder_midpoint_pos[1] - 5 - shoulder_diff_text_size[1] - 2)
        shoulder_diff_box_end = (shoulder_diff_box_start[0] + shoulder_diff_text_size[0] + 4, shoulder_midpoint_pos[1] - 5 + 2)
        cv2.rectangle(image, shoulder_diff_box_start, shoulder_diff_box_end, (255, 255, 255), cv2.FILLED)
        cv2.rectangle(image, shoulder_diff_box_start, shoulder_diff_box_end, (230, 216, 173), 1)

        # Deviation Box
        deviation_box_start = (central_vertical_line_pixel_x + 5, shoulder_midpoint_pos[1] + 15 - deviation_text_size[1] - 2)
        deviation_box_end = (deviation_box_start[0] + deviation_text_size[0] + 4, shoulder_midpoint_pos[1] + 15 + 2)
        cv2.rectangle(image, deviation_box_start, deviation_box_end, (255, 255, 255), cv2.FILLED)
        cv2.rectangle(image, deviation_box_start, deviation_box_end, (230, 216, 173), 1)

        # Now put the text on top of the boxes
        cv2.putText(image, shoulder_diff_text, (shoulder_diff_box_start[0], shoulder_diff_box_start[1] + shoulder_diff_text_size[1] + 2), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (139, 0, 0), 1)
        cv2.putText(image, deviation_full_text, (deviation_box_start[0], deviation_box_start[1] + deviation_text_size[1] + 2), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (139, 0, 0), 1)
        
        
        # Add text for posture
        cv2.putText(image, f'Posture: {posture}', (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2)

    return image


## 3.1 Stand front

### 3.1.1 Analysis

In [10]:
def stand_front_analysis(landmarks, mp_pose):
    """
    Analyzes front-view standing posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing the detected posture and a tuple of angle differences for ears, shoulders, hips, knees, and ankles.
    """
    # Get points for ears, shoulders, hips, knees, and ankles
    left_ear, left_shoulder, left_hip, left_knee, left_ankle = map(
        lambda lm: get_point(landmarks, lm),
        [mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.LEFT_SHOULDER, 
         mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, 
         mp_pose.PoseLandmark.LEFT_ANKLE])

    right_ear, right_shoulder, right_hip, right_knee, right_ankle = map(
        lambda lm: get_point(landmarks, lm),
        [mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.RIGHT_SHOULDER, 
         mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE, 
         mp_pose.PoseLandmark.RIGHT_ANKLE])

    # Calculate joint angles
    left_ear_angle = abs(calculate_angle([left_ear[0], 0], left_ear, right_ear))
    right_ear_angle = abs(calculate_angle([right_ear[0], 0], right_ear, left_ear))

    left_shoulder_angle = abs(calculate_angle([left_shoulder[0], 0], left_shoulder, right_shoulder))
    right_shoulder_angle = abs(calculate_angle([right_shoulder[0], 0], right_shoulder, left_shoulder))

    left_hip_angle = abs(calculate_angle([left_hip[0], 0], left_hip, right_hip))
    right_hip_angle = abs(calculate_angle([right_hip[0], 0], right_hip, left_hip))

    left_knee_angle = abs(calculate_angle([left_knee[0], 0], left_knee, right_knee))
    right_knee_angle = abs(calculate_angle([right_knee[0], 0], right_knee, left_knee))

    left_ankle_angle = abs(calculate_angle([left_ankle[0], 0], left_ankle, right_ankle))
    right_ankle_angle = abs(calculate_angle([right_ankle[0], 0], right_ankle, left_ankle))

    # Determine if joints are level using angle differences and buffer

    ear_postures = ["H.L. ear", "H.R. ear"]
    shoulder_postures = ["H.L. shoulder", "H.R. shoulder"]
    hip_postures = ["H.L. hip", "H.R. hip"]
    knee_postures = ["H.L. knee", "H.R. knee"]
    ankle_postures = ["H.L. ankle", "H.R. ankle"]

    posture = ""
    if left_ear_angle - right_ear_angle > sml_buffer:
        posture += sign + ear_postures[0]
    elif right_ear_angle - left_ear_angle > sml_buffer:
        posture += sign + ear_postures[1]

    if left_shoulder_angle - right_shoulder_angle > sml_buffer:
        posture += sign + shoulder_postures[0]
    elif right_shoulder_angle - left_shoulder_angle > sml_buffer:
        posture += sign + shoulder_postures[1]

    if left_hip_angle - right_hip_angle > sml_buffer:
        posture += sign + hip_postures[0]
    elif right_hip_angle - left_hip_angle > sml_buffer:
        posture += sign + hip_postures[1]

    if left_knee_angle - right_knee_angle > sml_buffer:
        posture += sign + knee_postures[0]
    elif right_knee_angle - left_knee_angle > sml_buffer:
        posture += sign + knee_postures[1]

    if left_ankle_angle - right_ankle_angle > sml_buffer:
        posture += sign + ankle_postures[0]
    elif right_ankle_angle - left_ankle_angle > sml_buffer:
        posture += sign + ankle_postures[1]

    if posture == "":
        posture = "Correct"

    # Update posture data
    detected_postures = posture.split(sign)
    for detected_posture in detected_postures:
        detected_posture = detected_posture.strip()
        if detected_posture in ear_postures:
            update_posture_data("Ear", ear_postures, detected_posture)
        elif detected_posture in shoulder_postures:
            update_posture_data("Shoulder", shoulder_postures, detected_posture)
        elif detected_posture in hip_postures:
            update_posture_data("Hip", hip_postures, detected_posture)
        elif detected_posture in knee_postures:
            update_posture_data("Knee", knee_postures, detected_posture)
        elif detected_posture in ankle_postures:
            update_posture_data("Ankle", ankle_postures, detected_posture)
        elif detected_posture == "Correct":
            posture_data["Correct"] = posture_data.get("Correct", 0) + 1

    return posture, (left_ear_angle - right_ear_angle, left_shoulder_angle - right_shoulder_angle, left_hip_angle - right_hip_angle, left_knee_angle - right_knee_angle, left_ankle_angle - right_ankle_angle)

# Define or import calculate_angle, posture_data, update_posture_data, mp_pose, and pose_landmark before using this function.


### 3.1.2 Detection

In [11]:
def stand_front_detection(image, results, posture, angles):
    """
    Analyzes and annotates an image for stand front exercises.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - posture (str): Description of the overall posture.
    - angles (tuple): Contains analysis data like shoulder vertical difference,
                                shoulder midpoint, and central vertical line position.

    Returns:
    - numpy.ndarray: The annotated image with landmarks, vertical line, and deviation information.
    """
    
    if results.pose_landmarks:
        landmark_idx = [7, 8, 11, 12, 23, 24, 25, 26, 27, 28]
        # Draw the lines
        for i in range(0, len(landmark_idx), 2):
            draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[i]], landmarks_to_draw[landmark_idx[i+1]])
        for i in range(2, len(landmark_idx)-2, 2):
            draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[i]], landmarks_to_draw[landmark_idx[i+2]], color=(0, 0, 255))
        for i in range(3, len(landmark_idx)-2, 2):
            draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[i]], landmarks_to_draw[landmark_idx[i+2]], color=(0, 0, 255))

        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[0]], landmarks_to_draw[landmark_idx[2]], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[1]], landmarks_to_draw[landmark_idx[3]], color=(0, 0, 255))
        draw_landmarks(image, results, landmark_idx) # Draw the specified landmarks
        
        joint_landmarks = [landmarks_to_draw[idx].value for idx in landmark_idx[::2]] # Draw the lines and display angles
        draw_labeled_box(image, results, joint_landmarks, angles) # Call the draw_labeled_box function
        
        # Add text for posture
        cv2.putText(image, f'Posture: {posture}', (0, 20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

    return image

## 3.2 Stand Side

### Analysis

In [12]:
def stand_side_analysis(landmarks, mp_pose):
    """
    Analyzes side-view standing posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing the detected posture and the left shoulder angle.
    """

    # Get points for ear and shoulder
    left_ear, left_shoulder, left_hip, left_knee, left_ankle = map(
        lambda lm: get_point(landmarks, lm),
        [mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.LEFT_SHOULDER, 
         mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE])

    # Posture Analysis
    # Calculate shoulder angle
    left_shoulder_angle = abs(calculate_angle([left_shoulder[0], 0], left_shoulder, left_ear))
    
    left_knee_angle = abs(calculate_angle(left_hip, left_knee, left_ankle))

    # Determine if shoulder angle is correct
    correct_shoulder_angle = 0  # Correct shoulder angle (ideal posture)
    shoulder_high = left_shoulder_angle > (correct_shoulder_angle + buffer)

    # Update posture condition
    shoulder_postures = ["Forward head"]
    posture = ""
    if shoulder_high:
        posture += sign + shoulder_postures[0]


    if posture == "":
        posture = "Correct"

    # Update posture data
    detected_posture = posture.strip(sign)
    if detected_posture in shoulder_postures:
        update_posture_data("Head", shoulder_postures, detected_posture)
    elif detected_posture == "Correct":
        posture_data["Correct"] = posture_data.get("Correct", 0) + 1

    return posture, (left_shoulder_angle, left_knee_angle)

### 3.2.2 Detection

In [13]:
def stand_side_detection(image, results, posture, angles):
    """
    Analyzes and annotates a side-view image of a standing posture.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - posture (str): Description of the overall posture.
    - angles (tuple): Tuple containing angles, where angles[0] is the neck angle and angles[1] is another angle, such as the knee angle.

    Returns:
    - numpy.ndarray: The annotated image with neck angle and posture information.
    """
    if results.pose_landmarks:
        landmark_idx = [7, 11, 23, 25, 27]
        # Draw the lines
        draw_colored_connection(image, results, landmarks_to_draw[7], landmarks_to_draw[11])
        draw_colored_connection(image, results, landmarks_to_draw[11], landmarks_to_draw[23], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[23], landmarks_to_draw[25])
        draw_colored_connection(image, results, landmarks_to_draw[25], landmarks_to_draw[27], color=(0, 0, 255))
        
        # Draw a vertical line across the left hip
        left_hip_landmark = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP.value]
        left_hip_x = int(left_hip_landmark.x * image.shape[1])
        cv2.line(image, (left_hip_x, 0), (left_hip_x, image.shape[0]), (255, 255, 0), thickness=1)
        
        
        draw_landmarks(image, results, landmark_idx) # Draw circles for specified landmarks
            
        joint_landmarks = [landmarks_to_draw[11].value, landmarks_to_draw[25].value] # Display angle information using a loop
        draw_labeled_box(image, results, joint_landmarks, angles) # Call the draw_labeled_box function

        # Add text for posture
        cv2.putText(image, f'Posture: {posture}', (10, 20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2)

    return image


# Run Main Function

In [19]:
if __name__ == "__main__":
    main()

SystemExit: 0

C:\Users\cheun\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
